# West Nile Virus

## Feature Engineering

In [7]:
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
import pandas.core.algorithms as algos
from pandas import Series
import scipy.stats.stats as stats
import re
import traceback
import string
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [8]:
df = pd.read_csv('/Users/ultimateflexer/Desktop/Datasets/predict-west-nile-virus/clean_df.csv')

In [9]:
df.head(10)

,Date,Address,Street,Trap,Latitude,Longitude,NumMosquitos,WnvPresent,Species_CULEX ERRATICUS,Species_CULEX PIPIENS,...,ResultSpeed_lag_14,ResultSpeed_lag_21,ResultDir_lag_1,ResultDir_lag_7,ResultDir_lag_14,ResultDir_lag_21,AvgSpeed_lag_1,AvgSpeed_lag_7,AvgSpeed_lag_14,AvgSpeed_lag_21
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",N OAK PARK AVE,T002,41.954690,-87.800991,1,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",N OAK PARK AVE,T002,41.954690,-87.800991,1,0,0,0,...,NaN,NaN,17.5,NaN,NaN,NaN,9.0,NaN,NaN,NaN
2,2007-05-29,"6200 North Mandell Avenue, Chicago, IL 60646, USA",N MANDELL AVE,T007,41.994991,-87.769279,1,0,0,0,...,NaN,NaN,17.5,NaN,NaN,NaN,9.0,NaN,NaN,NaN
3,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",W FOSTER AVE,T015,41.974089,-87.824812,1,0,0,0,...,NaN,NaN,17.5,NaN,NaN,NaN,9.0,NaN,NaN,NaN
4,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",W FOSTER AVE,T015,41.974089,-87.824812,4,0,0,0,...,NaN,NaN,17.5,NaN,NaN,NaN,9.0,NaN,NaN,NaN
5,2007-05-29,"1500 West Webster Avenue, Chicago, IL 60614, USA",W WEBSTER AVE,T045,41.921600,-87.666455,2,0,0,0,...,NaN,NaN,17.5,NaN,NaN,NaN,9.0,NaN,NaN,NaN
6,2007-05-29,"2500 West Grand Avenue, Chicago, IL 60654, USA",W GRAND AVE,T046,41.891118,-87.654491,1,0,0,0,...,NaN,NaN,17.5,NaN,NaN,NaN,9.0,NaN,NaN,NaN
7,2007-05-29,"1100 Roosevelt Road, Chicago, IL 60608, USA",W ROOSEVELT,T048,41.867108,-87.654224,1,0,0,0,...,NaN,NaN,17.5,17.5,NaN,NaN,9.0,9.0,NaN,NaN
8,2007-05-29,"1100 Roosevelt Road, Chicago, IL 60608, USA",W ROOSEVELT,T048,41.867108,-87.654224,2,0,0,0,...,NaN,NaN,17.5,17.5,NaN,NaN,9.0,9.0,NaN,NaN
9,2007-05-29,"1100 West Chicago Avenue, Chicago, IL 60642, USA",W CHICAGO,T049,41.896282,-87.655232,1,0,0,0,...,NaN,NaN,17.5,17.5,NaN,NaN,9.0,9.0,NaN,NaN


In [10]:
# Before I perform information value analysis on the data, I am going to drop the address 
# columns as well as the trap column. I also need to deal with NaN values. These will be dropped
# so as not to interfere with the lag variables. 

df.set_index('Date', inplace=True)
df.drop(['Address', 'Street', 'Trap'], axis=1, inplace=True)

df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10392 entries, 2007-05-29 to 2013-09-26
Data columns (total 87 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Latitude                        10392 non-null  float64
 1   Longitude                       10392 non-null  float64
 2   NumMosquitos                    10392 non-null  int64  
 3   WnvPresent                      10392 non-null  int64  
 4   Species_CULEX ERRATICUS         10392 non-null  int64  
 5   Species_CULEX PIPIENS           10392 non-null  int64  
 6   Species_CULEX PIPIENS/RESTUANS  10392 non-null  int64  
 7   Species_CULEX RESTUANS          10392 non-null  int64  
 8   Species_CULEX SALINARIUS        10392 non-null  int64  
 9   Species_CULEX TARSALIS          10392 non-null  int64  
 10  Species_CULEX TERRITANS         10392 non-null  int64  
 11  Tmax                            10392 non-null  float64
 12  Tmin                   

In [11]:
# Split data into train and test sets

y = df['WnvPresent']
X = df.drop('WnvPresent', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, 
                                                    random_state=42, stratify=y)

In [ ]:
# Information Value Analysis via the Variance Inlfation Factor.

def iterate_vif(df, vif_threshold=5, max_vif=6):
    count = 0
    while max_vif > vif_threshold:
        count += 1
        print("Iteration # "+str(count))
        vif = pd.DataFrame()
        vif["VIFactor"] = [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]
        vif["features"] = df.columns

        if vif['VIFactor'].max() > vif_threshold:
            print('Removing %s with VIF of %f' % (vif[vif['VIFactor'] == vif['VIFactor'].max()]['features'].values[0], vif['VIFactor'].max()))
            df = df.drop(vif[vif['VIFactor'] == vif['VIFactor'].max()]['features'].values[0], axis=1)
            max_vif = vif['VIFactor'].max()
        else:
            print('Complete')
    return df, vif.sort_values('VIFactor')

final_df, final_vif = iterate_vif(X_train)

Iteration # 1


/opt/anaconda3/lib/python3.8/site-packages/statsmodels/stats/outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


Removing Species_CULEX ERRATICUS with VIF of inf
Iteration # 2


/opt/anaconda3/lib/python3.8/site-packages/statsmodels/stats/outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


Removing Tavg with VIF of inf
Iteration # 3


...

/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/opt/anaconda3/lib/python3.8/site-packages/statsmodels/stats/outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Removing Tavg_lag_7 with VIF of inf
Iteration # 5


/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
/opt/anaconda3/lib/python3.8/site-packages/statsmodels/stats/outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


Removing Tavg_lag_14 with VIF of inf
Iteration # 6


/opt/anaconda3/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


import proper module ^^ from stats model 

,Latitude,Longitude,NumMosquitos,Species_CULEX ERRATICUS,Species_CULEX PIPIENS,Species_CULEX PIPIENS/RESTUANS,Species_CULEX RESTUANS,Species_CULEX SALINARIUS,Species_CULEX TARSALIS,Species_CULEX TERRITANS,...,Trap_T230,Trap_T231,Trap_T232,Trap_T233,Trap_T235,Trap_T236,Trap_T237,Trap_T238,Trap_T900,Trap_T903
Date,,,,,,,,,,,,,,,,,,,,,
2007-05-29,41.954690,-87.800991,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2007-05-29,41.954690,-87.800991,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2007-05-29,41.994991,-87.769279,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2007-05-29,41.974089,-87.824812,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2007-05-29,41.974089,-87.824812,4,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
